In [1]:
using DataFrames, DelimitedFiles, DataStructures
using GLPKMathProgInterface
using JuMP, StatsBase, DataFrames, CSV, LinearAlgebra, Distributions, Random
using SCIP, GLPK,BenchmarkTools

In [145]:
function clean_input_v2(file)
    #cd("/Users/kevin/Desktop/Draftkings/2022/Data/ETR/")
    offense = CSV.read(file, copycols=true, DataFrame)

    
    rename!(offense, Symbol(string("DK Salary")) => :Salary)
    rename!(offense, Symbol(string("DK Ceiling")) => :Projection)
    rename!(offense, Symbol(string("DK Ownership")) => :Ownership)




    offense = offense[completecases(offense), :]

 
    #cut off players that have projected points less than 5
    offense = offense[(offense."Projection".>=9) .| (offense."DK Position".== "DST"),:]
    #remove injured players

    #scores_o = copy(offense)
    #global scores_d = copy(defense)

    global num_offense = size(offense)[1]

    
    QB_indicies = Vector{Int}()
    RB_indicies = Vector{Int}()
    WR_indicies = Vector{Int}()
    TE_indicies = Vector{Int}()
    FLEX_indicies = Vector{Int}()
    DST_indicies = Vector{Int}()


   
    for i =1:num_offense
        if offense."DK Position"[i] == "QB"
            append!(QB_indicies, i)
        elseif offense."DK Position"[i] == "RB"
            append!(RB_indicies, i)
            append!(FLEX_indicies, i)
        elseif offense."DK Position"[i] == "WR"
            append!(WR_indicies, i)
            append!(FLEX_indicies, i)
        elseif offense."DK Position"[i] == "TE"
            append!(TE_indicies, i)
            append!(FLEX_indicies, i)
        elseif offense."DK Position"[i] == "DST"
            append!(DST_indicies, i)
        else
            append!(FLEX_indicies, i)
        end
    end
    
    return offense, QB_indicies, RB_indicies, WR_indicies, TE_indicies, FLEX_indicies, DST_indicies
        
        
    
end

clean_input_v2 (generic function with 2 methods)

In [3]:
function calc_cor(player_matrix, temp_teams, player_cor, i, j)
    if ismissing(temp_teams[i,2])
        player_matrix[i,j] .= 0
    elseif ismissing(temp_teams[j,2])
        player_matrix[i,j] = 0
    elseif temp_teams[i,2]==temp_teams[j,2]
        try
            orig_a = player_cor[:,i]
            orig_b = player_cor[:,j]

            a = orig_a[(ismissing.(orig_b).==false) .& (ismissing.(orig_a).==false),1]
            b = orig_b[(ismissing.(orig_a).==false) .& (ismissing.(orig_b).==false),1]
            
            if (i ==1) & (j == 47)
                println(a, b)
            end
            player_matrix[i,j] = cor(a, b)
        catch err
            player_matrix[i,j] = 0
        end
    else
        player_matrix[i,j] = 0
    end
    return player_matrix[i,j]
end

calc_cor (generic function with 1 method)

In [4]:
function calc_var(player_matrix, temp_teams, player_cor, i)

    orig_a = player_cor[:,i]
    #a = orig_a[(ismissing.(orig_a).==false),1]
    a = skipmissing(orig_a)

    
    println(a)
    try
        variance = var(a)
    catch err
        variance = 0    
    end
    
    return variance
end

calc_var (generic function with 1 method)

In [5]:
function get_variance(from, to, v, max_score)
    #cd("/Users/kevin/Dropbox (MIT)/Draft Kings/2021/2020SeasonData/")
    offense2 = get_stripped_offense(from)
    offense2 = offense2[:,[:Player_Name,:Actual_FP]]
    temp = offense2
    
    for i=from+1:to-1
        offense2 = get_stripped_offense(i)
        offense2 = offense2[:,[:Player_Name,:Actual_FP]]

   
        temp = outerjoin(temp, offense2, on = :Player_Name, makeunique=true)
        
    end
    #print(temp)
    
    #temp = Tables.matrix(temp[:,2:end])
    #print(temp)
    #print(temp)
    #cd("/Users/kevin/Dropbox (MIT)/Draft Kings/2021/2020SeasonData/")
    current_offense, QB_i, RB_i, WR_i, TE_i, FLEX_i, DST_i = clean_input_v2(to, v);
    current_offense."Player" .= replace.(current_offense."Player", r"-" => "")
    current_offense."Player" .= replace.(current_offense."Player", r"\." => "")
    current_offense."Player" .= replace.(current_offense."Player", r"'" => "")
    transform!(current_offense, :Player => ByRow(lowercase) => :Player)
    rename!(current_offense, Symbol(string("Player")) => :Player_Name)
   
   #print(current_offense) 
    
    var_array = []
    
    for i=1:length(current_offense[:,1])
        points = []
        #println(current_offense[i,1])
        #println(Matrix(filter(row -> row.Player_Name == current_offense[i,1], temp)))
        
        #println(temp)
        
        if current_offense[i,1] in temp[:,1]
            #println(current_offense[i,1])
            points_df = filter(row -> row.Player_Name == current_offense[i,1], temp)
            #println(points_df)
            for j = 2:(to-from)+1
                if ismissing(points_df[1,j]) == false
                    if Float64.(points_df[1,j]) != 0
                        append!(points, points_df[1,j])
                    end
                end
            end
            
            #println(current_offense[i,1], points)
        else            
            points = 0
        end
        
        if length(points) > 0
            variance = var(points)
        else
            variance = 0
        end
        
        if variance > 0
            append!(var_array, variance)
        else
            append!(var_array, 0)
        end
            
        #println(var_array)
    end
    
    #println(var_array)

    b = Float64.(var_array)
    
    scale = maximum(b) / max_score
    
    b = b ./ scale
    
    #print(b)
    
    return b
end



get_variance (generic function with 1 method)

In [6]:
function get_stripped_offense(week)
    cd("/Users/kevin/Desktop/Draftkings/2022/Data/DFN_Historical/")
    offense = CSV.read(string("DFN NFL Offense DK Week ",week,".csv"), copycols=true, DataFrame)
    rename!(offense, Symbol(string("Proj FP")) => :Projection)
    #offense = offense[offense[:,:Projection].>=3.01,:]
    
    offense."Player Name" .= replace.(offense."Player Name", r"-" => "")
    offense."Player Name" .= replace.(offense."Player Name", r"\." => "")
    offense."Player Name" .= replace.(offense."Player Name", r"'" => "")
    
    rename!(offense, Symbol(string("Player Name")) => :Player_Name)
    rename!(offense, Symbol(string("Actual FP")) => :Actual_FP)
    transform!(offense, :Player_Name => ByRow(lowercase) => :Player_Name)

    return offense
end

get_stripped_offense (generic function with 1 method)

In [7]:
function get_correlation_matrix(from, to, v)
    #cd("/Users/kevin/Dropbox (MIT)/Draft Kings/2021/2020SeasonData/")

    for i=from:to-1
        offense2 = get_stripped_offense(i)
        offense2 = offense2[:,[:Player_Name,:Actual_FP]]


        #print(i)
        if i==from
            temp = offense2
        else
            temp = outerjoin(temp, offense2, on = :Player_Name, makeunique=true)
        end
    end
    #print(temp)
    #cd("/Users/kevin/Dropbox (MIT)/Draft Kings/2021/2020SeasonData/")
    current_offense, QB_i, RB_i, WR_i, TE_i, FLEX_i, DST_i = clean_input_v2(to, v);
    current_offense."Player" .= replace.(current_offense."Player", r"-" => "")
    current_offense."Player" .= replace.(current_offense."Player", r"\." => "")
    current_offense."Player" .= replace.(current_offense."Player", r"'" => "")
    transform!(current_offense, :Player => ByRow(lowercase) => :Player)
    rename!(current_offense, Symbol(string("Player")) => :Player_Name)


    current_offense = current_offense[:,[:Player_Name,:Team]]
    #temp = temp[:,[:Player_Name,:Actual_FP]]

    #print(temp)
    #print(current_offense)
    #print(temp)
    #gold skeleton key
    temp = leftjoin(current_offense, temp, on = :Player_Name, makeunique=true)


    #temp = leftjoin(current_offense, temp, on = [:Player_Name])
    #print(temp2)
    #println(temp)
    temp = temp[:,setdiff(names(temp), [:Team])]
    temp_teams = current_offense

    temp = Matrix(temp)
    #print(temp)
    temp = permutedims(temp)
    #print(temp)


    player_list = temp[1,:]
    player_cor = temp[3:end,:]
    temp_teams = temp_teams[:,[:Player_Name,:Team]]
    #println(player_cor)



    n = size(temp_teams)[1]
    player_matrix = zeros(n,n)
    #temp_teams
    for i=1:n,j=1:n
        
        #if i in FLEX_i && j in FLEX_i && j in opp_team[i]
         #   player_matrix[i,j] = (cOffense[j,:Projection]/(9*10))
        if i in DST_i || j in DST_i
            player_matrix[i,j] = 0
        elseif i in QB_i && j in RB_i
            player_matrix[i,j] = calc_cor(player_matrix, temp_teams, player_cor, i, j)
            if player_matrix[i,j] < -0.2
                player_matrix[i,j] = -100
            else
                player_matrix[i,j] = player_matrix[i,j] * 0.25
            end
        elseif j in QB_i && i in RB_i
            player_matrix[i,j] = calc_cor(player_matrix, temp_teams, player_cor, i, j) 
            if player_matrix[i,j] < -0.2
                player_matrix[i,j] = -100
            else
                player_matrix[i,j] = player_matrix[i,j] * 0.25
            end
        elseif i in QB_i || j in QB_i
            player_matrix[i,j] = calc_cor(player_matrix, temp_teams, player_cor, i, j)
        else
            player_matrix[i,j] = calc_cor(player_matrix, temp_teams, player_cor, i, j)
            if player_matrix[i,j] < -0.2
                player_matrix[i,j] = -100
            else
                player_matrix[i,j] = player_matrix[i,j] * 0.25
            end
        end
        if isnan(player_matrix[i,j])
            player_matrix[i,j] = 0
        end
    end

    for i=1:n
        player_matrix[i,i] = 1
    end
    player_matrix[player_matrix.<-0.2] .= -100;
    return player_matrix
end

get_correlation_matrix (generic function with 1 method)

In [8]:
function get_bring_back(week, QB_i, offense)
    
    bring_back = Vector{Vector{Int}}()

    for i = 1:length(QB_i)
        temp = Vector{Int}()
        for j = 1:size(t)[1]
            
            if (offense[j, :Team] == offense[QB_i[i], :Opponent]) && (j in FLEX_i)
                append!(temp, j)

            end
        end
        push!(bring_back, temp)   
    end
    
    return bring_back
end

get_bring_back (generic function with 1 method)

In [9]:
function get_opposing_defense(week, QB_i, offense)
    
    opp_defense = Vector{Vector{Int}}()

    for i = 1:length(QB_i)
        temp = Vector{Int}()
        for j = 1:size(t)[1]
            
            if (offense[QB_i[i], :Team] == offense[j, :Opponent]) && (j in DST_i)
                append!(temp, j)

            end
        end
        push!(opp_defense, temp)   
    end
    
    return opp_defense
end

get_opposing_defense (generic function with 1 method)

In [10]:
function get_low_ownership_ids(ownership_thresh, offense, DST_i)
    temp = Vector{Int}()

    for i = 1:length(offense[:,1])
            
        if (offense[i, :Ownership] <= ownership_thresh) && (i ∉ DST_i)
            append!(temp, i)
        end

    end
    
    return temp
end



get_low_ownership_ids (generic function with 1 method)

In [11]:
function get_opposing_team(week, offense)
    
    opp_team = Vector{Vector{Int}}()

    for i = 1:size(t)[1]
        temp = Vector{Int}()
        for j = 1:size(t)[1]
            
            if (offense[i, :Team] == offense[j, :Opponent]) && i in FLEX_i && j in FLEX_i
                append!(temp, j)

            end
        end
        push!(opp_team, temp)   
    end
    
    return opp_team
end

get_opposing_team (generic function with 1 method)

In [12]:
function get_games(df)
    unq = unique(df[:,:Team])
    num_games = length(unq)
    game = Vector{Vector{Int}}()

    for i = 1:num_games
        temp = Vector{Int}()
        for j = 1:size(t)[1]
            
            if j in FLEX_i || j in DST_i
                if (df[j, :Team] == unq[i]) || (df[j, :Opponent] == unq[i]) 
                    append!(temp, j)

                end
            end
        end
        push!(game, temp)   
    end
    
    return game, num_games
    
end

get_games (generic function with 1 method)

In [13]:
function get_teams(df)
    unq = unique(df[:,:Team])
    num_teams = length(unq)
    team = Vector{Vector{Int}}()

    for i = 1:num_teams
        temp = Vector{Int}()
        for j = 1:size(t)[1]
            
            #if j in FLEX_i || j in DST_i
                if (df[j, :Team] == unq[i]) 
                    append!(temp, j)

                end
           # end
        end
        push!(team, temp)   
    end
    
    return team, num_teams
    
end

get_teams (generic function with 1 method)

In [14]:
function get_idx(t)
    team_idx = Vector{Vector{Int}}()

    for i=1:size(t)[1]

        temp = Vector{Int}()

        for j=1:size(t)[2]
            if i<j
                if t[i,j] != 0
                    append!(temp, j)
                end
            end    
        end

        push!(team_idx, temp)   

    end




    #####################

    team_idx_cross = Vector{Vector{Int}}()

    for i=1:size(t)[1]

        temp = Vector{Int}()

        for j=1:size(t)[2]
            #if i<=j
                if t[i,j] > 0
                    append!(temp, j)
                end
            #end    
        end

        push!(team_idx_cross, temp)   
    end
    
    return team_idx, team_idx_cross
end

get_idx (generic function with 1 method)

In [15]:
function late_swap_ids(week, new_version, late_teams)
    df, QB_i, RB_i, WR_i, TE_i, FLEX_i, DST_i = clean_input_v2(week, new_version)

    ids = readdlm(string("/Users/kevin/Desktop/Draftkings/2022/Data/ETR/Week ", week, "/2022_Week_1_ETR_IDS_2.csv"), ',', Int)
    
    all_teams = []
    for i = 1:length(ids[:,1])
        team_i = []
        for j = 1:9            
            row = findall(x -> x == ids[i,j], df."DKSlateID")
            if length(row) > 0
                if (df[row[1], :Team] in late_teams) == false 
                    team_i = push!(team_i, row[1])
                end
            end
        end
        all_teams = push!(all_teams, team_i)
    end
    
    return all_teams
end

late_swap_ids (generic function with 1 method)

In [ ]:
#function (standings, DK_entries)
    #order by standings

#end

In [123]:
standings = "/Users/kevin/Desktop/Draftkings/2022/Data/ETR/Week1/contest-standings-week1v1.csv"

username = "Highmaintinance"

function open_standings(standings, username)
    st = CSV.read(standings, copycols=true, DataFrame, select=[1,2,3,4,5])
    
    filteredDf = st[startswith.(st[:,:EntryName],username),:]


    return filteredDf
    
end

open_standings (generic function with 1 method)

In [124]:
st = open_standings(standings, username);

┌ Warning: thread = 1 warning: only found 9 / 11 columns around data row: 559. Filling remaining columns with `missing`
└ @ CSV /Users/kevin/.julia/packages/CSV/h66c0/src/file.jl:620
┌ Warning: thread = 1 warning: only found 9 / 11 columns around data row: 560. Filling remaining columns with `missing`
└ @ CSV /Users/kevin/.julia/packages/CSV/h66c0/src/file.jl:620
┌ Warning: thread = 1 warning: only found 9 / 11 columns around data row: 561. Filling remaining columns with `missing`
└ @ CSV /Users/kevin/.julia/packages/CSV/h66c0/src/file.jl:620
┌ Warning: thread = 1 warning: only found 9 / 11 columns around data row: 562. Filling remaining columns with `missing`
└ @ CSV /Users/kevin/.julia/packages/CSV/h66c0/src/file.jl:620
┌ Warning: thread = 1 warning: only found 9 / 11 columns around data row: 563. Filling remaining columns with `missing`
└ @ CSV /Users/kevin/.julia/packages/CSV/h66c0/src/file.jl:620
┌ Warning: thread = 1 warning: only found 9 / 11 columns around data row: 564. Fillin

In [136]:
entries = "/Users/kevin/Desktop/Draftkings/2022/Data/ETR/Week1/DKEntries_week1v2.csv"

contest = 129401843

function open_entries(entries, contest)
    ent = CSV.read(entries, copycols=true, DataFrame, select=[1,3,4,5,6,7,8,9,10,11,12,13])
    
    
    
    ent = dropmissing(ent, :QB)
    
    
    filteredDf = ent[(ent."Contest ID".==contest),:]

    rename!(filteredDf, Symbol(string("Entry ID")) => :EntryId)



    return filteredDf
    
end

open_entries (generic function with 2 methods)

In [137]:
ent = open_entries(entries, contest)
st = open_standings(standings, username);

┌ Warning: thread = 1 warning: only found 9 / 11 columns around data row: 559. Filling remaining columns with `missing`
└ @ CSV /Users/kevin/.julia/packages/CSV/h66c0/src/file.jl:620
┌ Warning: thread = 1 warning: only found 9 / 11 columns around data row: 560. Filling remaining columns with `missing`
└ @ CSV /Users/kevin/.julia/packages/CSV/h66c0/src/file.jl:620
┌ Warning: thread = 1 warning: only found 9 / 11 columns around data row: 561. Filling remaining columns with `missing`
└ @ CSV /Users/kevin/.julia/packages/CSV/h66c0/src/file.jl:620
┌ Warning: thread = 1 warning: only found 9 / 11 columns around data row: 562. Filling remaining columns with `missing`
└ @ CSV /Users/kevin/.julia/packages/CSV/h66c0/src/file.jl:620
┌ Warning: thread = 1 warning: only found 9 / 11 columns around data row: 563. Filling remaining columns with `missing`
└ @ CSV /Users/kevin/.julia/packages/CSV/h66c0/src/file.jl:620
┌ Warning: thread = 1 warning: only found 9 / 11 columns around data row: 564. Fillin

In [138]:
df = leftjoin(ent, st, on = :EntryId)

,EntryId,Contest ID,Entry Fee,QB,RB,RB_1,WR,WR_1
,Int64?,Int64?,String7?,String31,String31?,String31?,String31?,String31?
1,3353898685,129401843,$5,23485911,23486000,23486056,23486398,23486418
2,3353898682,129401843,$5,23485911,23486012,23486052,23486378,23486398
3,3353898802,129401843,$5,23485918,23486012,23486034,23486418,23486382
4,3353898719,129401843,$5,23485911,23486016,23486056,23486378,23486418
5,3353898690,129401843,$5,23485911,23486012,23486034,23486418,23486564
6,3353898819,129401843,$5,23485911,23486012,23486034,23486398,23486388
7,3353898728,129401843,$5,23485911,23486034,23486058,23486378,23486388
8,3353898798,129401843,$5,23485911,23486004,23486056,23486388,23486396
9,3353898815,129401843,$5,23485911,23486004,23486016,23486398,23486432


In [152]:
df

,EntryId,Contest ID,Entry Fee,QB,RB,RB_1,WR,WR_1
,Int64?,Int64?,String7?,String31,String31?,String31?,String31?,String31?
1,3353898685,129401843,$5,23485911,23486000,23486056,23486398,23486418
2,3353898682,129401843,$5,23485911,23486012,23486052,23486378,23486398
3,3353898802,129401843,$5,23485918,23486012,23486034,23486418,23486382
4,3353898719,129401843,$5,23485911,23486016,23486056,23486378,23486418
5,3353898690,129401843,$5,23485911,23486012,23486034,23486418,23486564
6,3353898819,129401843,$5,23485911,23486012,23486034,23486398,23486388
7,3353898728,129401843,$5,23485911,23486034,23486058,23486378,23486388
8,3353898798,129401843,$5,23485911,23486004,23486056,23486388,23486396
9,3353898815,129401843,$5,23485911,23486004,23486016,23486398,23486432


In [143]:
late_teams = ["GB", "MIN", "NYG", "TEN", "LV", "LAC", "KC", "ARI"]

8-element Vector{String}:
 "GB"
 "MIN"
 "NYG"
 "TEN"
 "LV"
 "LAC"
 "KC"
 "ARI"

In [157]:
function late_swap_ids(late_teams)
    
    file = "/Users/kevin/Desktop/Draftkings/2022/Data/ETR/Week1/Week_1_6.csv"


    x, QB_i, RB_i, WR_i, TE_i, FLEX_i, DST_i = clean_input_v2(file)
    
    all_teams = []
    for i = 1:length(df[:,1])
        team_i = []
        for j = 1:9            
                if (x[row[1], :Team] in late_teams) == false 
                    team_i = push!(team_i, row[1])
                end
        end
        all_teams = push!(all_teams, team_i)
    end
    
    return all_teams
end

late_swap_ids (generic function with 2 methods)

In [158]:
x = late_swap_ids(late_teams)

150-element Vector{Any}:
 Any[]
 Any[]
 Any[]
 Any[]
 Any[]
 Any[]
 Any[]
 Any[]
 Any[]
 Any[]
 Any[]
 Any[]
 Any[]
 ⋮
 Any[]
 Any[]
 Any[]
 Any[]
 Any[]
 Any[]
 Any[]
 Any[]
 Any[]
 Any[]
 Any[]
 Any[]

In [118]:
function lateSwapTeamX(num, X, cOffense, cNum_offense, QB_i, RB_i, WR_i, TE_i, FLEX_i, num_QB, num_RB, num_WR, num_TE, num_FLEX, bring_back, team_idx, team_idx_cross, opp_team, game_stack, num_games, rand_vec, team_stack, num_teams, current_teams, late_games)#num is what team it is from 1-100, X is current teams
    m1 = Model(SCIP.Optimizer)
    set_silent(m1)
    #GLPK.Optimizer
    M = 100
    max_overlap = 5.1
    max_same_player = 40
    @variable(m1, x[1:cNum_offense], Bin) #pick players
    #@variable(m1, def[1:cNum_teams], Bin) #pick players
    #@variable(m1, PB[1:cNum_offense], Bin) #player boost
    
    
    #@constraint(m1, [i=1:length(FLEX_i)], PB[FLEX_i[i]] <= sum(x[opp_team[i][j]] for j = 1:length(opp_team[i])))
    
    #@constraint(m1, [i=1:length(QB_i)], PB[QB_i[i]] == 0)
    #@constraint(m1, [i=1:length(DST_i)], PB[DST_i[i]] == 0)

    #@variable(m1, G[1:num_games], Bin)
    #@constraint(m1,[i=1:num_games], 1.5 * G[i] <= sum(x[game_stack[i][j]] for j=1:length(game_stack[i])))
    #@constraint(m1, sum(G[i] for i=1:num_games) >= 2)
    #need to make num_games and game_stack, try using uniques to get it
    
    @variable(m1, u) #pick players


    @objective(m1, Max, u)

    @constraint(m1, sum(x[i]*rand_vec[i] for i=1:cNum_offense) +  # cOffense[i, :Projection] for i=1:cNum_offense) + 
                    (sum(t[i,team_idx[i][j]]*x[i]*x[team_idx[i][j]] for i=1:cNum_offense for j=1:length(team_idx[i])) * 9)>= u)
                    #sum(0.1 * x[i] * x[opp_def[i][j]] for i=1:cNum_offense for j=1:length(opp_def[i])) +
                    #sum(0.1*PB[i]*cOffense[i, :Projection] for i=1:cNum_offense) +
                    

    @constraint(m1, sum(x[QB_i[i]] for i=1:num_QB) == 1)
    @constraint(m1, sum(x[RB_i[i]] for i=1:num_RB) >= 2)
    @constraint(m1, sum(x[WR_i[i]] for i=1:num_WR) >= 3)
    @constraint(m1, sum(x[TE_i[i]] for i=1:num_TE) >= 1)
    @constraint(m1, sum(x[DST_i[i]] for i=1:num_DST) == 1)
    @constraint(m1, sum(x[FLEX_i[i]] for i=1:num_FLEX) == 7)
    @constraint(m1,sum(x[i]*cOffense[i, :Salary] for i=1:cNum_offense) <= 50000)
    @constraint(m1, sum(x[i] for i=1:cNum_offense) == 9)
                        
    @constraint(m1,sum(x[i]*cOffense."DK Ownership"[i] for i=1:cNum_offense) <= 50)
                        
    for i=1:cNum_offense
        #println(i)
        if (cOffense[i, :Team] in late_games) == false
            if isempty(current_teams[num]) == false
                #println(num)
                if i in current_teams[num]
                    #println(current_teams[num])


                    @constraint(m1, x[i] == 1)
                else
                    @constraint(m1, x[i] == 0)            
                end
            end
        end
    end


    
    
    #@constraint(m1, sum(t[i,team_idx[i][j]]*x[i]*x[team_idx[i][j]] for i=1:cNum_offense for j=1:length(team_idx[i])) >= 1.5)
    
    #bring back
                    
    @constraint(m1,[i=1:length(QB_i)], x[QB_i[i]] - sum(x[bring_back[i][j]] for j = 1:length(bring_back[i])) <= 0)
                        
    #team stack          
    @constraint(m1,[i=1:num_teams], sum(x[team_stack[i][j]] for j=1:length(team_stack[i])) <= 4)
    
    @constraint(m1,[i=1:num_teams], sum(x[team_stack[i][j]] for j=1:length(team_stack[i])) <= 4)



    
    #@constraint(m1, sum(def[i] for i=1:cNum_teams) == 1)


    if num != 1
        #for i=1:num
        @constraint(m1,[i=1:num-1], sum(t[X[i][k],team_idx_cross[X[i][k]][j]] * x[team_idx_cross[X[i][k]][j]] for k = 1:9 for j=1:length(team_idx_cross[X[i][k]])) <= 6.9)
        #end
                        
        #@constraint(m1,[i=1:num-1], sum(x[X[i][k]] for k = 1:8) <= max_overlap)


    end 
                                        
    optimize!(m1)
    return(JuMP.value.(x))
end

lateSwapTeamX (generic function with 1 method)

In [119]:
function lateSwapAllTeams(count, X, cOffense, cNum_offense, QB_i, RB_i, WR_i, TE_i, FLEX_i, num_QB, num_RB, num_WR, num_TE, num_FLEX, bring_back, team_idx, team_idx_cross, opp_team, game_stack, num_games, team_stack, num_teams, current_teams, late_games) #count is how many teams you want
    
    rand_vec = rand(cNum_offense,1)/25 .+1
    temp = cOffense[:, :Projection]
    rand_vec = rand_vec .* temp
    x1 = lateSwapTeamX(1, X, cOffense, cNum_offense, QB_i, RB_i, WR_i, TE_i, FLEX_i, num_QB, num_RB, num_WR, num_TE, num_FLEX, bring_back, team_idx, team_idx_cross, opp_team, game_stack, num_games, rand_vec, team_stack, num_teams, current_teams, late_games);

    B = Vector{Int}()
    for i=1:length(x1)
        if x1[i] > 0.5
            append!(B,i)
        end
    end
    X = push!(X, B)
    
    for i = 2:count
        rand_vec = rand(cNum_offense,1)/25 .+1
        temp = cOffense[:, :Projection]
        rand_vec = rand_vec .* temp
        @time x1 = lateSwapTeamX(i, X, cOffense, cNum_offense, QB_i, RB_i, WR_i, TE_i, FLEX_i, num_QB, num_RB, num_WR, num_TE, num_FLEX, bring_back, team_idx, team_idx_cross, opp_team, game_stack, num_games, rand_vec, team_stack, num_teams, current_teams, late_games);
        B = Vector{Int}()
        for i=1:length(x1)
            if x1[i] > 0.5
                append!(B,i)
            end
        end
        
        X = push!(X, B)
        #print(X)
        #D = hcat(D, d1)
        
        println(i)

    end

    return(X)
end

lateSwapAllTeams (generic function with 1 method)